# Decision Tree


**Name**: María Gabriela Ayala

In [ ]:
import math
import pickle
import gzip
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
%matplotlib inline

### Problem 1 - Decision tree
***

Consider the problem of predicting whether a person has a college degree based on age, salary, and Chicago residency. 
The dataset looks like the following.

| Age   | Salary         | Chicago Residency      | College degree| 
|:------:|:------------:| :-----------:|---:|
| 27 | 41,000 | Yes | Yes |
| 61 | 52,000 | No | No |
| 23 | 24,000 | Yes | No |
| 29 | 77,000 | Yes | Yes |
| 32 | 48,000 | No | Yes |
| 57 | 120,000 | Yes | Yes |
| 22 | 38,000 | Yes | Yes |
| 41 | 45,000 | Yes | No |
| 53 | 26,000 | No | No |
| 48 | 65,000 | Yes | Yes |


**Part A [5 points]**: Convert the above table to data. Two variables should be created:
        
1. $x$ is a $10*3$ matrix that contains the data from columns 0, 1, and 2. Chicago residency is represented by 1 (yes) and 0 (no).
2. $y$ contains the labels (college degree), 1 (yes) and 0 (no).

In [ ]:
x = np.array([[27, 41000, 1], [61, 52000, 0], [23, 24000, 1], [29, 77000, 1], [32, 48000, 0], [57, 120000, 1], [22, 38000, 1], [41, 45000, 1], [53, 26000, 0], [48, 65000, 1]])
y = np.array([[1], [0], [0], [1], [1], [1], [1], [0], [0], [1]])
print("x =", x)
print("y =", y)

**Part B:** Criteria for choosing a feature to split.

We start with no splitting. Assuming that our algorithm is deterministic, what is the smallest number of mistakes we can make if we do not use any of the features and what is the algorithm? (**Write your answer in the Markdown cell below.**)

If we don't use any of the features, then with no splitting the algorithm would predict "Yes"/1 for college degree because there are 6 entries labeled 1 and 4 labeled 0. This means that the smallest number of mistakes we can make without using any of the features is 4, which corresponds to the 4 entries labeled as "No"/0 that would incorrectly be classified as "Yes"/1.
The algorithm would count the instances of 0 and 1 labels in the y matrix and determine the max value as the default prediction label.

We start by considering the variable *Chicago residency*. The first criteria is based on the number of mistakes. We need to build a contingency table between Chicago residency and college degree.

How many mistakes will we make if we split based on Chicago residency? (**Answer below by finishing the code.**)

In [ ]:


def get_error_in_leaf(y, ids):
    """
    Returns the errors in a leaf node of a decision tree.
    This function can be used to answer the previous question automatically.
    
    :@param y: all labels
    :@param ids: the subset of indexes in the leaf node
    """
    ids = list(range((len(y))))
    count_0 = 0
    count_1 = 0

    for i in ids:
        if y[i][0] == 0:
            count_0 +=1
        elif y[i][0] == 1:
            count_1 += 1
    return min(count_0, count_1)


def error_criteria(y, root, left_child, right_child):
    """
    Returns the number of errors if we split the root into the left child and the right child.
    
    :@param y: all labels
    :@param root: indexes of all the data points in the root
    :@param left_child: the subset of indexes in the left child
    :@param right_child: the subset of indexes in the right child
    """

    left_0 = 0
    left_1 = 0

    for lidx in left_child:
        if y[root[lidx]][0] == 0:
            left_0 +=1
        elif y[root[lidx]][0] == 1:
            left_1 += 1
    left_error = min(left_0, left_1)

    right_0 = 0
    right_1 = 0

    for ridx in right_child:
        if y[root[ridx]][0] == 0:
            right_0 +=1
        elif y[root[ridx]][0] == 1:
            right_1 += 1
    right_error = min(right_0, right_1)

    return left_error + right_error



def value_split_binary_feature(x, y, fid, root, criteria_func):
    
    root = root.tolist()
    left_child = [i for i in root if x[i, fid] == 0]
    right_child = [i for i in root if x[i, fid] == 1]
    return criteria_func(y, root, left_child, right_child)

# Chicago residency should correspond to the third column in your data x
fid = 2
root = np.array(list(range(len(y)))) # root includes all data points
mistakes = value_split_binary_feature(x, y, fid, root, error_criteria)

3

**Part C** Alternatively, we can use entropy and information gain to split the data. In this part, you will manually determine the first split in a decision tree. Please do not use code in your calculations for part C.

To get familiar with MathJax, please write the equations necessary to compute entropy and information gain if we split data $D$ into $D_1$ and $D_2$. **Write your answer in the Markdown cell below.**

[1] Split the data using entropy:
$$H(X) = -p\space log_2p - (1 - p)log_2(1 - p)$$
where:
$p = \frac{D_1}{D}$ assuming $D_1$ is the positive class, ie. 1 in a binary 1, 0 or 1, -1 relationship.
$\newline$
Here:
$\newline$
$0\leq p \leq 1$

[2] Split the data using information gain, based on feature i:

$\newline$
Where:
$\newline$

$$X_{parent} = X_{D}$$
$$X_{i,left} = X_{D_1, left}$$
$$X_{i,right} = X_{D_2, right}$$

The information gain formula is:
$$IG(X_{D,i}) = H(X_{D}) - \frac{|X_{i,D_1}|}{|X_{D}|} H(X_{D_1}) - \frac{|X_{i,D_2}|}{|X_{D}|} H(X_{D_2})


**[4 points]** What is the entropy for College Degree? Please show your work. Were you expecting a result around this number? Why?

$$p = \frac{6}{10} = \frac{3}{5}$$
$$H(X) = -p\space log_2 p - (1 - p)log_2(1 - p)$$
$$H(X) = -\frac{3}{5}log_2\frac{3}{5} - (1 - \frac{3}{5})log_2(1 - \frac{3}{5})$$
$$H(X) = -\frac{3}{5} \space log_2\frac{3}{5} - (\frac{2}{5})log_2(\frac{2}{5})$$
$$H(X) = 0.9709505944546686 $$

I was expecting a number close to 1, since the labels for college degree are almost balanced, with 6 entries classified as "Yes"/1 and 4 entries classified as "No"/0. Out of a total of 10, 6-4 is very close to a perfect balance 5-5, therefore the result matches my expectations.

**[4 points]** What is the information gain for College Degree if you split the observations on the Chicago Residency attribute? Please show your work.

$$X_{parent} = X_{D} = \{1, 0, 0, 1, 1, 1, 1, 0, 0, 1\}$$
$$X_{i,left} = X_{Chicago Residency, left} = \{0, 1, 0\} $$
$$X_{i,right} = X_{Chicago Residency, right} = \{1, 0, 1, 1, 1, 0, 1\}$$

Entropy for $X_{parent}$:
$$H(X_{parent}) = 0.9709505944546686 $$

Entropy for $X_{Chicago Residency, left}$:
$$p = \frac{1}{3}$$
$$H(X_{Chicago Residency, left}) = -p\space log_2 p - (1 - p)log_2(1 - p)$$
$$H(X_{Chicago Residency, left})  = -\frac{1}{3}\space log_2\frac{1}{3} - (1 - \frac{1}{3})log_2(1 - \frac{1}{3})$$
$$H(X_{Chicago Residency, left})  = -\frac{1}{3} \space log_2\frac{1}{3} - (\frac{2}{3})log_2(\frac{2}{3})$$
$$H(X_{Chicago Residency, left}) = 0.9182958340544896 $$

Entropy for $X_{Chicago Residency, right}$:
$$p = \frac{5}{7}$$
$$H(X_{Chicago Residency, right}) = -p\space log_2 p - (1 - p)log_2(1 - p)$$
$$H(X_{Chicago Residency, right})  = -\frac{5}{7}\space log_2\frac{5}{7} - (1 - \frac{5}{7})log_2(1 - \frac{5}{7})$$
$$H(X_{Chicago Residency, right})  = -\frac{5}{7} \space log_2\frac{5}{7} - (\frac{2}{7})log_2(\frac{2}{7})$$
$$H(X_{Chicago Residency, right}) = 0.863120568566631 $$


Information Gain calculation:
$$IG(X,Chicago Residency) = H(X_{parent}) - \frac{|X_{Chicago Residency,left}|}{|X_{parent}|} H(X_{Chicago Residency, left}) - \frac{|X_{Chicago Residency, right}|}{|X_{parent}|} H(X_{Chicago Residency, right})$$
$$IG(X,Chicago Residency) = 0.9709505944546686 - (3/10)* 0.9182958340544896  - (7/10)* 0.863120568566631$$
$$IG(X,Chicago Residency) = 0.0912774462416801$$






One way to deal with continuous (or ordinal) features like Age and Salary is to define binary features based on thresholding. For example, you might convert ages to 0 if Age is less than or equal to 50 and 1 otherwise. What is the information gain for College Degree if we split the observations based on the Salary attribute with a threshold of \$50,000? Please show your work.

(Later in this problem, we will write code to determine whether a number other than \$50,000 might be the optimal threshold for Salary. For now, just assume that a threshold of \\$50,000 is optimal.)

Updating the Salary feature with a threshold of $50,000, we get the following table:
| Age   | Salary         | Chicago Residency      | College degree| 
|:------:|:------------:| :-----------:|---:|
| 27 | 0 | Yes | 1 |
| 61 | 1 | No | 0 |
| 23 | 0 | Yes | 0 |
| 29 | 1 | Yes | 1 |
| 32 | 0 | No | 1 |
| 57 | 1 | Yes | 1 |
| 22 | 0| Yes | 1 |
| 41 | 0| Yes | 0 |
| 53 | 0 | No | 0 |
| 48 | 1| Yes | 1 |

To calculate the information gain for College Degree splitting on the Salary attribute:
$$X_{parent} = X_{D} = \{1, 0, 0, 1, 1, 1, 1, 0, 0, 1\}$$
$$X_{i,left} = X_{Salary, left} = \{1, 0, 1, 1, 0, 0\} $$
$$X_{i,right} = X_{Salary, right} = \{0, 1, 1, 1\}$$

Entropy for $X_{parent}$:
$$H(X_{parent}) = 0.9709505944546686 $$

Entropy for $X_{Salary, left}$:
$$p = \frac{3}{6} = \frac{1}{2} $$
$$H(X_{Salary, left}) = -p\space log_2 p - (1 - p)log_2(1 - p)$$
$$H(X_{Salary, left})  = -\frac{1}{2}\space log_2\frac{1}{2} - (1 - \frac{1}{2})log_2(1 - \frac{1}{2})$$
$$H(X_{Salary, left})  = -\frac{1}{2} \space log_2\frac{1}{2} - (\frac{1}{2})log_2(\frac{1}{2})$$
$$H(X_{Salary, left}) = 1.0 $$

Entropy for $X_{Salary, right}$:
$$p = \frac{3}{4}$$
$$H(X_{Salary, right}) = -p\space log_2 p - (1 - p)log_2(1 - p)$$
$$H(X_{Salary, right})  = -\frac{3}{4}\space log_2\frac{3}{4} - (1 - \frac{3}{4})log_2(1 - \frac{3}{4})$$
$$H(X_{Salary, right})  = -\frac{3}{4} \space log_2\frac{3}{4} - (\frac{1}{4})log_2(\frac{1}{4})$$
$$H(X_{Salary, right}) =  0.8112781244591328 $$

Information Gain calculation:
$$IG(X,Salary) = H(X_{parent}) - \frac{|X_{Salary,left}|}{|X_{parent}|} H(X_{Salary, left}) - \frac{|X_{Salary, right}|}{|X_{parent}|} H(X_{Salary, right})$$
$$IG(X,Salary) = 0.9709505944546686 - (6/10)* 1.0 - (4/10)* 0.8112781244591328$$
$$IG(X,Salary) = 0.04643934467101546$$



Based on the information gain calculations, should the first split in our decision tree be on Chicago Residency or on Salary with a \$50,000 threshold? Is this the answer you expected based on the data counts?

Based on the information gain calculations, the first split in the decision tree should be Chicago Residency because it yields a higher information gain than splitting on Salary: 0.091 > 0.046.
This is the answer I expected based on the data counts, since the combined entropy for Chicago Residency is lower than that for Salary. The left and right child data for Chicago Residency is split between 1/3 and 5/7 whereas it is split between 3/6 and 3/4 for Salary, such that it is more imbalanced for Chicago Residency and therefore there is more to be learned by splitting first by this attribute.

**Part D** Now we will write a function for computing information gain. Use log2 for entropy computation.

In [ ]:
def entropy(y, ids):
    """
    Returns the entropy in the labels for the data points in ids.
    
    :@param y: all labels
    :@param ids: the indexes of data points
    """
    if len(ids) == 0: # deal with corner case when there is no data point.
        return 0
    
    count_1 = 0 
    for i in ids:
        if y[i][0] == 1:
            count_1 += 1
    
    p = count_1/len(y)

    return -(p)*math.log2(p) - (1-p)*math.log2(1-p)

    
def information_gain_criteria(y, root, left_child, right_child):
    """
    Returns the information gain by splitting root into left child and right child.
    
    :@param y: all labels
    :@param root: indexes of all the data points in the root
    :@param left_child: the subset of indexes in the left child
    :@param right_child: the subset of indexes in the right child
    """
    ids = list(range((len(y))))
    parent_entropy = entropy(y, ids)

    if len(left_child) == 0 or len(right_child) == 0:
        return 0

    left_1 = 0

    for lidx in left_child:
        if y[root[lidx]][0] == 1:
            left_1 += 1

    p_left = left_1/len(left_child)
    if p_left == 0 or p_left == 1:
        return 0
    else:
        left_entropy = -(p_left)*math.log2(p_left) - (1-p_left)*math.log2(1-p_left)

    right_1 = 0

    for ridx in right_child:
        if y[root[ridx]][0] == 1:
            right_1 += 1
    p_right = right_1/len(right_child)
    if p_right == 0 or p_right == 1:
        return 0
    else:
        right_entropy = -(p_right)*math.log2(p_right) - (1-p_right)*math.log2(1-p_right)


    return  parent_entropy - len(left_child)/len(y)* left_entropy  - len(right_child)/len(y)* right_entropy
    
fid = 2
root = np.array(list(range(len(y)))) # root includes all data points
info_gain = value_split_binary_feature(x, y, fid, root, information_gain_criteria)   

0.0912774462416801

In [ ]:
# This cell is for grading purposes only; please ignore

**Part E**: Deal with continuous features.
    
Complete the following code:

In [ ]:
def value_split_continuous_feature(x, y, fid, root, criteria_func=information_gain_criteria):
    """
    Return the best value and its corresponding threshold by splitting based on a continuous feature.

    :@param x: all feature values
    :@param y: all labels
    :@param fid: feature id to split the tree based on
    :@param root: indexes of all the data points in the root
    :@param criteria_func: the splitting criteria function
    """
    best_value, best_thres = 0, 0
    
    for i in range(len(x)):
        x1= np.copy(x)
        threshold = x1[root[i]][fid]
        for i in root:
            if x1[i][fid] > threshold:
                x1[i][fid] = 1
            else:
                x1[i][fid] = 0
        left_child = [i for i in root if x1[i][fid] == 0]
        right_child = [i for i in root if x1[i][fid] == 1]
        info_gain = information_gain_criteria(y, root, left_child, right_child)
        if info_gain > best_value:
            best_value = info_gain
            best_thres = threshold
         
    return best_value, best_thres

root = np.array(range(len(y))) # root includes all data points
fid = 0
age_value, age_thres = value_split_continuous_feature(x, y, fid, root, information_gain_criteria)
fid = 1
salary_value, salary_thres = value_split_continuous_feature(x, y, fid, root, information_gain_criteria)

Based on the current information gain by splitting different features, if we build a decision stump (decision tree with depth 1) greedily, which feature should we choose? Why? **Write down your answer in the Markdown cell below.**

From the continuous features analysis, the best threshold for age is at 32 with an information gain of 0.124, compared to a best threshold of $45.000 for salary the same information gain of 0.124. In both continuous variable cases, best threshold yields the exact same information gain. Alternatively, the previously calculated Chicago Residency yields an information gain of 0.091.
From a decision stump approach, either age or salary at their optimal respective thresholdcould be the first split since they yield the highest information gain.

**Extra credit**: You now have all the ingredients to build a decision tree recursively. You can build a decision tree of depth two and report its classification error on the training data and the tree.

In [ ]:
class LeafNode:
    """
    Class for leaf nodes in the decision tree
    """
    
    def __init__(self, label, count, total):
        """
        :@param label: label of the leaf node
        :@param count: number of data points with class 'label' falling in this leaf
        :@param count: number of datapoints of any label falling in this leaf
        """
        self.label = label
        self.count = count
        self.total = total
        
    def predict(self, x):
        """
        Return predictions for features x

        :@param x: feature values
        """
        # YOUR CODE HERE
        raise NotImplementedError()
    
    def display(self, feat_names, out_str, depth=0):
        """
        Display contents of a leaf node
        """
        prefix = '\t'*depth
        error = 1.0 - self.count / float(self.total)
        out_str += f'{prefix}leaf: label={self.label}, error={error} ({self.count}/{self.total} correct)\n'
        return out_str
    
class TreeNode:
    """
    Class for internal (non-leaf) nodes in the decision tree
    """
    def __init__(self, feat_id, feat_val):
        """
        :@param feat_id: index of the feature that this node splits on
        :@param feat_val: threshold for the feature that this node splits on
        """
        self.feat_id = feat_id
        self.feat_val = feat_val
        self.left = None
        self.right = None
    
    def split(self, x, root):
        """
        Given the datapoints falling into current node, return two arrays of indices in x corresponding to the
        left and right subtree
        
        :@param x: all feature values
        :@param root: indexes of all the data points in the current node
        """
        root = np.array(root)
        # YOUR CODE HERE
        raise NotImplementedError()
    
    def predict(self, x):
        """
        Return an array of predictions for given 'x' for the current node
        
        :@param x: datapoints
        """
        assert self.left is not None and self.right is not None, 'predict called before fit'
        # YOUR CODE HERE
        raise NotImplementedError()
    
    def display(self, feat_names, out_str, depth=0):
        """
        Display contents of a non-leaf node
        """
        prefix = '\t'*depth
        out_str += f'{prefix}{feat_names[self.feat_id]}\n'
        out_str += f'{prefix}x <= {self.feat_val}\n'
        out_str = self.left.display(feat_names, out_str, depth=depth+1)
        out_str += f'{prefix}x > {self.feat_val}\n'
        out_str = self.right.display(feat_names, out_str, depth=depth+1)
        return out_str

class DecisionTree:
    """
    Class for the decision tree
    """
    def __init__(self, max_depth=1, criteria_func=information_gain_criteria, binary_feat_ids=[]):
        """
        :@param max_depth: Maximum depth that a decision tree can take
        :@param criteria_func: criteria function to split features
        :@param binary_feat_id: list of indexes of binary features
        """
        self.max_depth = max_depth
        self.criteria_func = criteria_func
        self.binary_feat_ids = binary_feat_ids
        self.root = None
        self.x = None
        self.y = None
        
    def fit(self, x, y):
        """
        Fit a tree to the given dataset using a helper function
        """
        self.x = x
        self.y = y
        self.root = self.fit_helper(np.array(list(range(self.x.shape[0]))))
    
    def fit_helper(self, root, depth=1):
        """
        Recursive helper function for fitting a decision tree
        Returns a node (can be either LeafNode or TreeNode)
        
        :@param root: array of indices of datapoints which fall into the current node
        :@param depth: current depth of the tree being built 
        """
        
        """
        Strategy:
        1. If current partition is pure i.e. labels corresponding to all indices in root are the same
           OR the maximum depth has been reached, stop building the tree and return a LeafNode
        2. If not, find out the best feature to split on along with the threshold, create a TreeNode and 
           recursively call fit_helper on the two splits (You can assume the threshold for a binary feature 
           to be 0.5). Finally, return the current node 
        """
        
        # YOUR CODE HERE
        raise NotImplementedError()
    
    def predict(self, x):
        """
        Return predictions for a given dataset  
        """
        assert self.root is not None, 'fit not yet called'
        # YOUR CODE HERE
        raise NotImplementedError()
    
    
    def display(self, feat_names):
        assert self.root is not None, 'fit not yet called'
        out_str = ""
        out_str = self.root.display(feat_names, out_str)
        return out_str

In [ ]:
# This cell is for grading purposes only; please ignore
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score


### Crime Prediction 

Download [reported crime data for 2021](https://data.cityofchicago.org/Public-Safety/Crimes-2021/dwme-t96c) from the Chicago Open Data Portal (we already prepared it in the data folder, but would you to know where the source was).

**Part A**: Load the data.  We will only use three columns of the dataset: Primary Type, Latitude, and Longitude.
- Be sure to drop any observations that are missing latitude and/or longitude.
- To reduce run time, only keep incidents reported as one of the the four most common crime types ('THEFT', 'BATTERY', 'CRIMINAL DAMAGE', 'ASSAULT'). 
- Use the train_test_split function from Scikit-Learn to split the data into training and validation sets.  Set random_state=123 and test_size=0.2.
- Finally, explore the training and validation sets and answer the following questions: 
  - How many total observations are in the training set? 
  - How many total observations are in the validation set? 

In [ ]:
# Write code for answering the questions in Part A and then put your answer in the Markdown cell below.
df = pd.read_csv("crimes.csv")[['Primary Type', 'Latitude', 'Longitude']]
df = df.dropna(subset=['Latitude', 'Latitude'])
df = df.loc[(df['Primary Type'] == 'THEFT') | (df['Primary Type'] == 'BATTERY') |(df['Primary Type'] == 'CRIMINAL DAMAGE') | (df['Primary Type'] == 'ASSAULT')]
train, validate = train_test_split(df, test_size=0.2, random_state=123)
X_train = train[['Latitude', 'Longitude']]
X_train = X_train.to_numpy()
y_train = train[['Primary Type']]
y_train = y_train.to_numpy()
# Make sure to set each of the variables below to the correct value. Do not rename the variables.
N_training_examples = 99749
N_validation_examples = 24938

There are 99749 observations in the training set and 24938 observations in the validation/test set.